In [30]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

In [31]:
column_names = list(range(0,4))
column_names.append('Classification')

df = pd.read_csv("user-knowledge-modeling.data.txt", names = column_names) 
df

,0,1,2,3,Classification
0.00,0.00,0.00,0.00,0.00,very_low
0.08,0.08,0.10,0.24,0.90,High
0.06,0.06,0.05,0.25,0.33,Low
0.10,0.10,0.15,0.65,0.30,Middle
0.08,0.08,0.08,0.98,0.24,Low
...,...,...,...,...,...
0.61,0.78,0.69,0.92,0.58,High
0.78,0.61,0.71,0.19,0.60,Middle
0.54,0.82,0.71,0.29,0.77,High
0.50,0.75,0.81,0.61,0.26,Middle


In [32]:
df.dtypes

0                 float64
1                 float64
2                 float64
3                 float64
Classification     object
dtype: object

In [33]:
#find number of null entries to see if there is a need to clean the dataset
df.isnull().sum()
#df = df.dropna()

0                 0
1                 0
2                 0
3                 0
Classification    0
dtype: int64

In [34]:
df.nunique()

0                 89
1                 83
2                 80
3                 80
Classification     4
dtype: int64

In [35]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 

df['Classification'] = label_encoder.fit_transform(df['Classification'])
df

,0,1,2,3,Classification
0.00,0.00,0.00,0.00,0.00,3
0.08,0.08,0.10,0.24,0.90,0
0.06,0.06,0.05,0.25,0.33,1
0.10,0.10,0.15,0.65,0.30,2
0.08,0.08,0.08,0.98,0.24,1
...,...,...,...,...,...
0.61,0.78,0.69,0.92,0.58,0
0.78,0.61,0.71,0.19,0.60,2
0.54,0.82,0.71,0.29,0.77,0
0.50,0.75,0.81,0.61,0.26,2


In [36]:
X = df.drop('Classification', axis=1)
y = df['Classification']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [39]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

tf.random.set_seed(42)
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(128, activation='relu', input_shape = (4,)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='softmax'))

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy",
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state =42, shuffle=True)

sum_acc_score = 0
sum_prec_score = 0
sum_recall_score = 0
sum_f1_score = 0


for train_index, test_index in kf.split(X):
    X_train = pd.DataFrame()
    y_train = pd.DataFrame()

    X_test = pd.DataFrame()
    y_test = pd.DataFrame()
    
    for index in train_index:
        X_train = X_train.append(X.iloc[index])
        y_train = y_train.append(pd.Series(y.iloc[index]), ignore_index = True)
        
    for index in test_index:
        X_test = X_test.append(X.iloc[index])
        y_test = y_test.append(pd.Series(y.iloc[index]), ignore_index = True)

    train_X = tf.convert_to_tensor(X_train)
    test_X = tf.convert_to_tensor(X_test)

    #one hot encode y_train and y_test
    train_y = tf.keras.utils.to_categorical(y_train)
    test_y = tf.keras.utils.to_categorical(y_test)

            
    history = model.fit(train_X, train_y, epochs=50, validation_data = (test_X, test_y),callbacks=[callback])
    predictions = model.predict(test_X)
    
    predicted_classes = predictions.argmax(axis=1)
    test_y_classes = test_y.argmax(axis=1)
    
    sum_acc_score += accuracy_score(test_y_classes, predicted_classes)
    sum_prec_score += precision_score(test_y_classes, predicted_classes, average = 'micro')
    sum_recall_score += recall_score(test_y_classes, predicted_classes, average = 'micro')
    sum_f1_score += f1_score(test_y_classes, predicted_classes, average = 'micro')

Epoch 1/50
7/7 [==============================] - 1s 80ms/step - loss: 1.3418 - accuracy: 0.3252 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3135 - val_accuracy: 0.3846 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
7/7 [==============================] - 0s 8ms/step - loss: 1.2426 - accuracy: 0.4515 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.2686 - val_accuracy: 0.4231 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
7/7 [==============================] - 0s 9ms/step - loss: 1.1709 - accuracy: 0.5194 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.2332 - val_accuracy: 0.4038 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/50
7/7 [==============================] - 0s 8ms/step - loss: 1.1080 - accuracy: 0.5485 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.1977 - val_accuracy: 0.4423 - val_precision: 1.0000 - val_recall: 0.0192
Epoch 5/50
7/7 [==============================] - 0s 8ms/step -

Epoch 38/50
7/7 [==============================] - 0s 7ms/step - loss: 0.1633 - accuracy: 0.9660 - precision: 0.9660 - recall: 0.9660 - val_loss: 0.1850 - val_accuracy: 0.9423 - val_precision: 0.9423 - val_recall: 0.9423
Epoch 39/50
7/7 [==============================] - 0s 7ms/step - loss: 0.1595 - accuracy: 0.9612 - precision: 0.9612 - recall: 0.9612 - val_loss: 0.1855 - val_accuracy: 0.9231 - val_precision: 0.9231 - val_recall: 0.9231
Epoch 40/50
7/7 [==============================] - 0s 6ms/step - loss: 0.1510 - accuracy: 0.9709 - precision: 0.9709 - recall: 0.9709 - val_loss: 0.1860 - val_accuracy: 0.9615 - val_precision: 0.9615 - val_recall: 0.9615
Epoch 41/50
7/7 [==============================] - 0s 7ms/step - loss: 0.1526 - accuracy: 0.9660 - precision: 0.9660 - recall: 0.9660 - val_loss: 0.1938 - val_accuracy: 0.9038 - val_precision: 0.9038 - val_recall: 0.9038
Epoch 42/50
7/7 [==============================] - 0s 8ms/step - loss: 0.1538 - accuracy: 0.9612 - precision: 0.9612

Epoch 11/50
2/2 [==============================] - 0s 3ms/step
Epoch 1/50
7/7 [==============================] - 0s 17ms/step - loss: 0.0859 - accuracy: 0.9662 - precision: 0.9662 - recall: 0.9662 - val_loss: 0.1963 - val_accuracy: 0.9412 - val_precision: 0.9412 - val_recall: 0.9412
Epoch 2/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0792 - accuracy: 0.9758 - precision: 0.9758 - recall: 0.9758 - val_loss: 0.2274 - val_accuracy: 0.9412 - val_precision: 0.9412 - val_recall: 0.9412
Epoch 3/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0835 - accuracy: 0.9807 - precision: 0.9807 - recall: 0.9807 - val_loss: 0.2227 - val_accuracy: 0.9412 - val_precision: 0.9412 - val_recall: 0.9412
Epoch 4/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0810 - accuracy: 0.9710 - precision: 0.9710 - recall: 0.9710 - val_loss: 0.2046 - val_accuracy: 0.9412 - val_precision: 0.9412 - val_recall: 0.9412
Epoch 5/50
7/7 [==============================] - 0s 9ms

In [41]:
print('Average Accuracy Score: ', sum_acc_score/5)
print('Average Precision Score: ', sum_prec_score/5)
print('Average Recall Score: ', sum_recall_score/5)
print('Average F1-Score: ', sum_f1_score/5)



Average Accuracy Score:  0.9650075414781298
Average Precision Score:  0.9650075414781298
Average Recall Score:  0.9650075414781298
Average F1-Score:  0.9650075414781298
